In [ ]:
import numpy as np
import os

In [ ]:
cwd = os.getcwd()
xyz_dir = f"{cwd}/../XYZ"
os.chdir(f"{xyz_dir}")

dimer_name = "ethanol_ethanol"

atom1_index = 3
atom2_index = 12
molecule1_index1 = 1
molecule1_index2 = 9
molecule2_index1 = 10
molecule2_index2 = 18

input_file = f"{dimer_name}.xyz"
output_file = f"{dimer_name}.arc"
output_A_file = f"{dimer_name}_A.arc"
output_B_file = f"{dimer_name}_B.arc"

factor = [0.8, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2]

In [ ]:
def read_xyz(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    num_atoms = int(lines[0].strip()[0])
    atoms = []
    full_lines = []
    coordinates = []
    for line in lines[1:]:  # Skip first line
        parts = line.split()
        full_lines.append(line.strip())  # Store full line for output format
        atoms.append(parts[1])  # Element type
        coordinates.append([float(parts[2]), float(parts[3]), float(parts[4])])
    return np.array(atoms), np.array(coordinates), full_lines

def write_xyz(filename, full_lines, dimer_name, factor):
    with open(filename, 'a') as f:
        f.write(f"{len(full_lines):>6} {dimer_name}_{factor:.2f}\n")
        for line in full_lines:
            f.write(f"{line}\n")

def write_molecule_xyz(filename, full_lines, molecule_start, molecule_end, dimer_name, factor):
    molecule = full_lines[molecule_start - 1:molecule_end]
    
    # Create a mapping of old indices to new indices
    old_to_new = {int(line.split()[0]): i + 1 for i, line in enumerate(molecule)}
    
    # Renumber atoms and connectivity
    new_full_lines = []
    for i, line in enumerate(molecule):
        parts = line.split()
        new_connectivity = "".join(f"{old_to_new[int(p)]:>6}" for p in parts[6:] if int(p) in old_to_new)
        new_line = f"{i+1:>6}  {parts[1]:<2}   {parts[2]:>10}  {parts[3]:>10}  {parts[4]:>10}{parts[5]:>6}{new_connectivity}"
        new_full_lines.append(new_line)
    
    with open(filename, 'a') as f:
        f.write(f"{len(new_full_lines):>6} {dimer_name}_{factor:.2f}\n")
        for line in new_full_lines:
            f.write(f"{line}\n")

def translate_molecule(atoms, coordinates, atom1_index, atom2_index, molecule2_index1, molecule2_index2, factor):
    # Adjust indices to be 1-based
    atom1_index -= 1
    atom2_index -= 1
    molecule2_index1 -= 1
    
    # Get selected atoms vector
    A1, A2 = coordinates[atom1_index], coordinates[atom2_index]
    axis_vector = A2 - A1
    axis_distance = np.linalg.norm(axis_vector)
    axis_unit_vector = axis_vector / axis_distance
    
    # Compute new translation distance
    new_distance = axis_distance * factor
    translation_vector = axis_unit_vector * (new_distance - axis_distance)
    
    # Translate the second part of the molecule (assumed to start from atom2_index)
    translated_coords = coordinates.copy()
    for i in range(molecule2_index1, molecule2_index2):
        translated_coords[i] += translation_vector
    
    return translated_coords

In [ ]:
# Main execution

os.system(f"rm {output_file}")
os.system(f"rm {output_A_file}")
os.system(f"rm {output_B_file}")

for f in factor:
    atoms, coordinates, full_lines = read_xyz(input_file)
    translated_coordinates = translate_molecule(atoms, coordinates, atom1_index, atom2_index, molecule2_index1, molecule2_index2, f)
    
    # Update full_lines with new coordinates
    new_full_lines = []
    for i, line in enumerate(full_lines):
        parts = line.split()
        new_coords = translated_coordinates[i]
        new_line = f"{int(parts[0]):>6}  {parts[1]:<2}   {new_coords[0]:>10.6f}  {new_coords[1]:>10.6f}  {new_coords[2]:>10.6f}" + "".join(f"{item:>6}" for item in parts[5:])
        new_full_lines.append(new_line)
    write_xyz(output_file, new_full_lines, dimer_name, f)
    write_molecule_xyz(output_A_file, new_full_lines, molecule1_index1, molecule1_index2, dimer_name, f)
    write_molecule_xyz(output_B_file, new_full_lines, molecule2_index1, molecule2_index2, dimer_name, f)